In [12]:
import pandas_datareader as pdr
import numpy as np
import pandas as pd
import scipy.optimize 
from datetime import datetime,time,timedelta
from strategy1 import strategy1

In [13]:
factors = pd.read_csv("../../Factors/merge_Factors.csv")
factor_returns = pd.read_csv("../../Analysis/factors_returns_multi.csv")


In [14]:
factors

,Date,Index,Return,Momentum_1,Momentum_2,Momentum_3,Turnover,Volitility_1,Volitility_2,Volitility_3,...,industry_Communication Services,industry_Consumer Cyclical,industry_Consumer Defensive,industry_Energy,industry_Financial Services,industry_Healthcare,industry_Industrials,industry_Real Estate,industry_Technology,industry_Utilities
0,2021-01-04,ABCB,0.000788,0.292037,0.268440,1.337910,-0.405406,-0.774165,0.152428,-0.156877,...,0,0,0,0,1,0,0,0,0,0
1,2021-01-05,ABCB,0.026509,0.584899,0.282007,1.286126,-0.391646,-0.783078,0.047123,-0.154379,...,0,0,0,0,1,0,0,0,0,0
2,2021-01-06,ABCB,0.077218,1.091331,1.069403,1.382568,-0.370173,-0.027238,0.558188,-0.039128,...,0,0,0,0,1,0,0,0,0,0
3,2021-01-07,ABCB,0.030145,1.034280,1.092753,1.540811,-0.352361,0.033409,0.550544,-0.046228,...,0,0,0,0,1,0,0,0,0,0
4,2021-01-08,ABCB,-0.014977,0.963283,1.087409,1.372949,-0.350298,0.009898,0.344343,-0.039491,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228190,2022-10-25,ZYNE,0.092565,-1.597193,-1.941425,-1.773187,0.095649,1.776335,1.907331,1.519530,...,0,0,0,0,0,1,0,0,0,0
228191,2022-10-26,ZYNE,-0.025000,-1.975490,-1.867003,-1.751913,0.069974,1.745276,1.889380,1.539250,...,0,0,0,0,0,1,0,0,0,0
228192,2022-10-27,ZYNE,-0.021368,-1.932648,-1.903772,-1.703804,0.056728,1.405069,1.858137,1.478534,...,0,0,0,0,0,1,0,0,0,0
228193,2022-10-28,ZYNE,0.055313,-1.120505,-1.880914,-1.689206,0.023390,0.993889,1.859977,1.520113,...,0,0,0,0,0,1,0,0,0,0


In [15]:
date = "2022-10-03"
selected_stocks = strategy1(date)
factors = factors.loc[(factors["Date"]==date) & (factors["Index"].isin(selected_stocks))]
factor_returns = factor_returns.loc[factor_returns["Date"]==date]
factor_returns = factor_returns.iloc[:,1:]
stock_list = factors["Index"].tolist()
factors = factors.iloc[:, 3:]
factors = np.array(factors)
factor_returns = np.array(factor_returns)

In [16]:
stock_return = np.dot(factors, factor_returns.T).reshape(-1)
stock_list = np.array(stock_list)

In [17]:

from datetime import datetime 

In [18]:

def optimize(weights,tickers,start,end,calc_window,rf_rate,short=False,
             price_type='Close',clean='True'):
          
        '''
        weights; *type = list, initial guess weights
        
        start; input as str, data start date format 'YYYY-MM-DD' 
        end; input as str, data end date format 'YYYY-MM-DD'
        
        calc_window; input as str, calculation window for returns, 
        
        rf_rate; desired risk-free rate to benchmark against
        
        price_type; *input as str, default = 'Close', options = 'Adj Close', 
        'Close', 'High','Low','Open', 'Volume'
        
        short = bool, default = 'False', if true weight bounds -1 to 1
        
        clean; bool, default = 'False', if true, the function will drop NA rows
        '''
        start = datetime.strptime(start,'%Y-%m-%d')
        end = datetime.strptime(end,'%Y-%m-%d')
        
        prices = pdr.get_data_yahoo(tickers, start, end)[price_type]
        
        op = prices.pct_change(periods=int(calc_window))
        op.iloc[-1,:]=stock_return
        if clean == False:
            op
            
        else:
            op = op.dropna()
        
        vcv_matrix = np.cov(op.T)
        
        avg_return = op.mean()
        
        def sharpe(weights,vcv_matrix,avg_return):        
            
            var = np.dot(np.dot(weights,vcv_matrix),weights)
            
            ret =  np.dot(weights,avg_return)
            
            s_ratio = -(ret-rf_rate)/np.sqrt(var)
            
            return s_ratio
           
        
        sharpe_cons = ({'type':'eq','fun': lambda x: sum(x)-1})
        
        
        if short == False:
            sharpe_bnds = ((0,1),)*len(tickers)
        
        if short == True:
            sharpe_bnds = ((-1,1),)*len(tickers)
        
        optimal = scipy.optimize.minimize(sharpe,weights,bounds = sharpe_bnds,
                                 args = (vcv_matrix,avg_return),
                                 constraints = sharpe_cons)
        
        return optimal


In [20]:

weights=[0.1]*10
tickers=stock_list
#end为要预测仓位的那天
end='2022-10-03'
#start为end向前一个月
start='2022-09-03'
#不变，始终为1
calc_window='1'

In [22]:

# 确定无风险收益率
rf=pd.read_csv('daily-treasury-rates.csv',index_col=0)

#rf=rf.iloc[:,5]

rf.reset_index(inplace=True)

rf['Date']=[datetime.strptime(i,"%m/%d/%Y") for i in list(rf['Date'].values)]

start_time=datetime.strptime(start,"%Y-%m-%d")
end_time=datetime.strptime(end,"%Y-%m-%d")

rf=rf.set_index("Date")


for i in range(len(rf.index)):
    if list(rf.index)[i]-end_time<=timedelta(0):
        rfend_time=list(rf.index)[i]
        break
for j in range(len(rf.index)):
    if list(rf.index)[j]-start_time<=timedelta(0):
        rfstart_time=list(rf.index)[j]
        break

rf_rate=rf.loc[rfend_time:rfstart_time,'1 Yr'].mean()*0.01/360


In [23]:

optimize(weights,tickers,start,end,calc_window,rf_rate,short=True,price_type='Adj Close',clean='True')

     fun: -0.44935339399105306
     jac: array([0.26822124, 0.3093264 , 0.15039186, 0.26125344, 0.23906942,
       0.2393553 , 0.22948002, 0.23965044, 0.21792896, 0.23942446])
 message: 'Optimization terminated successfully'
    nfev: 210
     nit: 19
    njev: 19
  status: 0
 success: True
       x: array([-1.        , -1.        ,  1.        , -1.        ,  0.79064693,
       -0.58599813,  1.        ,  0.63867673,  1.        ,  0.15667447])

In [24]:
w=optimize(weights,tickers,start,end,calc_window,rf_rate,short=True,price_type='Adj Close',clean='True')
weight=list(w.x)